In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
from scipy import sparse

# Generate some indices
Even the sparse matrices won't fit in memory. So we will have to loop through them when making predictions or sampling random items.

In [2]:
#count number of items:
indptr = [0]

for chunkID in range(12):
    scores = np.load(f'../processed_data/D4_all{chunkID}.npy')
    indptr.append(indptr[-1] + scores.shape[0])


In [3]:
scores = np.concatenate([np.load(f'../processed_data/D4_all{i}.npy') for i in range(12)])

# functions to handle the slabs

For training, these loop through the chunks and extract the indices that have been selected either at random or suggested by the surrogate model. 

For predicting, these loop through the chunks and perform the `predict_proba` method on each chunk (after removing the training indices), outputting a concatenated numpy array of predicted values.

In [4]:
def extractFPs(chunkID, indptr, isTrain):
    fp = sparse.load_npz(f'../processed_data/D4_all{chunkID}.npz')
    mask = isTrain[indptr[chunkID]:indptr[chunkID+1]]
    return fp[mask]

def buildTrain(indptr, isTrain, verbose=0):
    if verbose:
        print('building training matrix')
    fps = sparse.vstack([extractFPs(i, indptr, isTrain) for i in range(12)])
    return fps

def chunkPredictProba(model, indptr, isTrain, verbose=0):
    if verbose:
        print('predicting probabilities')
    probas = []
    for chunkID in range(12):
        fps = extractFPs(chunkID, indptr, ~isTrain)
        proba = model.predict_proba(fps)[:,1]
        probas.append(proba)
    return np.concatenate(probas)

# Train and RF regressor and Logistic Regression models

In [5]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000, C=1)

In [6]:
topK = (scores.argsort().argsort() < (scores.shape[0]*0.0005)) #0.05th percentile.

#topK = (scores.argsort().argsort() < 50_000) #~0.05th percentile for AmpC, but not for D4
#tot = topK.sum() 

In [7]:
tot = topK.sum()

In [8]:
print(tot)

58121


# With Altair, using three repeats:

In [9]:

trainingSetSizes=[400_000, 200_000, 100_000]
#for percentile in [0.05, 0.1, 0.25, 0.5, 0.75, 1 ]:
for percentile in [0.3]:
    
    df = pd.DataFrame(columns=['Algorithm', 'Training size', 'N ligands explored', '% top-k found'])
    count=0
    
    for i in range(3):
        idx = np.arange(scores.shape[0])
        np.random.shuffle(idx)

        for size in trainingSetSizes:
            #split indices into train and test:
            train = idx[:size].copy()
            test = idx[size:].copy()
            train.sort()
            test.sort()
    
            #generate a 'is a training instance' mask. 
            isTrain = np.zeros(scores.shape[0]).astype(bool)
            isTrain[train]=True
    
            #topK molecules already found in the training set:
            numFound = topK[train].sum()
        
            df.loc[count] = ['morgan_feat', size, train.shape[0], numFound/tot]
            count+=1
            print(count, numFound)
            
            #estimate the cutoff once, from the initial random sample:
            cutoff = np.percentile(scores[train], percentile)
    
            for i in range(5):
                
                #fit model:
                model.fit(buildTrain(indptr, isTrain, 1), scores[isTrain]<cutoff)
    
                #predict (slowest step):
                proba = chunkPredictProba(model, indptr, isTrain, 1)
    
                #rank the probabilities
                proba_sorted = (-proba).argsort()
        
                #rank the unseen instances:
                test = test[proba_sorted]

                #now append the next N instances from the rank ordered unseen instances onto the training set:
                train = np.concatenate([train, test[:size]])
        
                #update the isTrain mask:
                isTrain[train]=True
        
                #now remove those training instances from the test set:
                test = test[size:]

                #keep the train and test idx arrays sorted so they agree with the chunked* methods:
                test.sort()
                train.sort()
        
                #topK molecules already found in the training set:
                numFound = topK[train].sum()
            
                df.loc[count] = ['morgan_feat', size, train.shape[0], numFound/tot]
                count+=1
                print(count, numFound)
                df.to_csv('../processed_data/D4_reconstruction_'+str(percentile)+'_1_.csv')
                
    df.to_csv('../processed_data/D4_reconstruction_'+str(percentile)+'_1_.csv')

1 197
building training matrix
predicting probabilities
2 14990
building training matrix
predicting probabilities
3 24587
building training matrix
predicting probabilities
4 31220
building training matrix
predicting probabilities
5 34899
building training matrix
predicting probabilities
6 37686
7 111
building training matrix
predicting probabilities
8 9032
building training matrix
predicting probabilities
9 16379
building training matrix
predicting probabilities
10 22244
building training matrix
predicting probabilities
11 26431
building training matrix
predicting probabilities
12 29556
13 47
building training matrix
predicting probabilities
14 4577
building training matrix
predicting probabilities
15 9727
building training matrix
predicting probabilities
16 14522
building training matrix
predicting probabilities
17 18300
building training matrix
predicting probabilities
18 21242
19 215
building training matrix
predicting probabilities
20 14618
building training matrix
predicting proba